In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cd ../input/dogs-vs-cats

In [ ]:
%ls

In [ ]:
%cd working

In [2]:
import zipfile
train_path = "../input/dogs-vs-cats/test.zip"

files = "Data"

with zipfile.ZipFile(train_path, 'r') as zipp:
    zipp.extractall(files)

In [3]:
import os
path_folder = 'Data/train/'
for file_name in os.listdir(path_folder)[:5]:
    image_path = path_folder + file_name
    print(image_path)
# print(len(os.listdir(path_folder)))

In [4]:
# import libraries
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import torchvision

In [5]:
class DogCatDataset(Dataset):
    """
    create dataset
    """

    def __init__(self, dataset_file, transform=None):
        self.dataset = pd.read_csv(dataset_file)
        self.transform = transform
    

    def __len__(self):
        return len(self.dataset)


    def __getitem__(self, index):
        image_path, label = self.dataset.iloc[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = Image.fromarray(image)
            image = self.transform(image)
            image = np.array(image)
        return image, label

In [6]:
class DogCatDataLoader:
    """
    load dataset --> return train and valid dataset
    """

    def __init__(self, dataset_file,
                       batch_size=8,
                       random_seed=42,
                       valid_size=0.2,
                       shuffle=True):
        self.dataset_file = dataset_file
        self.batch_size = batch_size
        self.random_seed = random_seed
        self.valid_size = valid_size
        self.shuffle = shuffle
    

    def create_data(self):
        normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                         std=[0.2023, 0.1994, 0.2010],)
        tranform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
            transforms.RandomAffine(degrees=40, translate=None, scale=(1, 2), shear=15, resample=False, fillcolor=0),
            transforms.ToTensor(),
            normalize
        ])

        train_dataset = DogCatDataset(dataset_file=self.dataset_file, transform=tranform) 
        valid_dataset = DogCatDataset(dataset_file=self.dataset_file, transform=tranform)

        num_train = len(train_dataset)
        indices = list(range(num_train))
        split = int(np.floor(self.valid_size * num_train))

        if self.shuffle:
            np.random.seed(self.random_seed)
            np.random.seed(indices)

        train_idx, valid_idx = indices[split:], indices[:split]
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        train_loader = DataLoader(
            dataset=train_dataset, batch_size=self.batch_size, sampler=train_sampler
        )

        valid_loader = DataLoader(
            dataset=valid_dataset, batch_size=self.batch_size, sampler=valid_sampler
        )

        return (train_loader, valid_loader)
#         return train_loader

In [11]:
class VGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes),
            nn.Softmax(dim=1))
    
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [28]:
dataset_folder = "Data/train/"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2
num_epochs = 10
batch_size = 16
learning_rate = 0.005
weight_decay = 0.005

# model = VGG16(num_classes).to(device)

model = torchvision.models.vgg16(pretrained=True)
model.classifier[-1] = nn.Linear(in_features=4096, out_features=2)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# dataset loader
dataset_file = "../input/cat-dog-recognition/train.csv"
data_loader = DogCatDataLoader(dataset_file=dataset_file, batch_size=batch_size)
train_loader, valid_loader = data_loader.create_data()
# train_loader = data_loader.create_data()

# Train the model
iterations = len(train_loader)

print("Start training...")
loss_opt = 1e9
for epoch in range(num_epochs):
    loss = None
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, iterations, loss.item()))
    if loss < loss_opt:
        loss_opt = loss
        torch.save(model.state_dict(), 'best.pth')
            
    # Validation
    with torch.no_grad(): 
        # turn off model.train() -> turn on model.eval() -> turn off model.eval() -> and then auto turn on model.train() again
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            outputs = F.softmax(outputs, dim=1)
#             print(outputs)
            _, predicted = torch.max(outputs.data, 1)
#             print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        print('Accuracy of the network on the validation images: {} %'.format(100 * correct / total)) 

In [63]:
check_point = './best.pth'
model_test = torchvision.models.vgg16()
model_test.classifier[-1] = nn.Linear(in_features=4096, out_features=2)
model_test.load_state_dict(torch.load(check_point))
model_test = model_test.to(device)
print(model_test)

In [64]:
image_path = './Data/test1/225.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                         std=[0.2023, 0.1994, 0.2010],)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

image = Image.fromarray(image)
image = transform(image)
# print(image.shape)
image = image.unsqueeze(0)
image = image.to(device)

model_test.eval()
output = model_test(image)
output = F.softmax(output, dim=1)
value_hat, label_hat = torch.max(output.data, 1)
print(label_hat)